# Simple result overview on TC displacement risk

This notebook gathers all results from the country folders and writes them to csv files for admin0 and admin1

In [9]:
import pandas as pd
import glob
import os
import numpy as np

path_results = '/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk/data/results/risk_tc/'# '/cluster/work/climate/evelynm/IDMC_UNU/results/risk_tc/'

In [10]:
all_iso3 = [cntry[0:3].upper() for cntry in os.listdir(path_results)]
len(all_iso3)

238

### Checking data for single countries

In [11]:
iso='AIA'

In [4]:
df_list=[]
for ssp in [245, 370, 585]:
    try:
        df_list.append(pd.read_csv(f'{path_results}{iso}/{iso}_ssp{ssp}_0.55_TC.csv', nrows=3))
    except:
        continue

In [5]:
cntry_df = pd.concat(df_list, axis=0).reset_index(drop=True)
cntry_df['iso3'] = iso
cntry_df.pop('Unnamed: 0')
cntry_df.pop('Exposure')
cntry_df

,Unnamed: 0.2,Unnamed: 0.1,Period,Scenario,AAD,RP_10,RP_25,RP_50,RP_100,RP_250,admin1,GID_1,iso3
0,0,0,hist,ERA-5,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
1,1,1,2050,ssp245,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
2,2,2,2100,ssp245,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
3,0,0,hist,ERA-5,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
4,1,1,2050,ssp370,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
5,2,2,2100,ssp370,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
6,0,0,hist,ERA-5,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
7,1,1,2050,ssp585,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
8,2,2,2100,ssp585,4.625209,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA


In [6]:
cntry_df.drop_duplicates(subset=['Period', 'Scenario', 'iso3'])

,Unnamed: 0.2,Unnamed: 0.1,Period,Scenario,AAD,RP_10,RP_25,RP_50,RP_100,RP_250,admin1,GID_1,iso3
0,0,0,hist,ERA-5,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
1,1,1,2050,ssp245,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
2,2,2,2100,ssp245,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
4,1,1,2050,ssp370,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
5,2,2,2100,ssp370,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
7,1,1,2050,ssp585,4.417873,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA
8,2,2,2100,ssp585,4.625209,0.0,0.0,0.0,0.0,0.0,-999,admin0,AIA


## Save all admin0 results

In [12]:
def process_threshold(thresh, all_iso3, path_results):
    """
    Process and aggregate results for a given threshold, and save the results to a CSV file.

    Parameters:
    - thresh: The threshold value to process (e.g., 0.3, 0.55, 0.7)
    - all_iso3: List of ISO3 country codes
    - path_results: The base path where the CSV files are located and results will be saved

    Returns:
    - None (saves the results to a CSV file)
    """
    global_list = []
    
    # Loop over each ISO3 code
    for iso in all_iso3:
        cntry_list = []
        
        # Check if the directory exists for the country
        if os.path.exists(path_results + iso):
            for ssp in [245, 370, 585]:
                try:
                    # Read the CSV file for the current ISO3, ssp, and threshold
                    cntry_list.append(pd.read_csv(f'{path_results}{iso}/{iso}_ssp{ssp}_{thresh}_TC.csv', nrows=3))
                except:
                    continue
            
            if cntry_list:
                # Concatenate the country-specific data and remove duplicates at the country level
                cntry_df = pd.concat(cntry_list, axis=0).reset_index(drop=True)
                cntry_df['iso3'] = iso
                
                # Drop unwanted columns
                cntry_df.pop('GID_1')
                cntry_df.pop('admin1')
                cntry_df.pop('Unnamed: 0')
                cntry_df.pop('Exposure')
                
                # Drop duplicates based on 'Period', 'Scenario', and 'iso3'
                cntry_df.drop_duplicates(subset=['Period', 'Scenario', 'iso3'], inplace=True)
                
                # Append the cleaned country dataframe to the global list
                global_list.append(cntry_df)
    
    # Concatenate all country dataframes into a global dataframe
    global_df = pd.concat(global_list, axis=0).reset_index(drop=True)
    
    # Drop duplicates at the global level to ensure no duplicates across countries or scenarios
    global_df.drop_duplicates(subset=['Period', 'Scenario', 'iso3'], inplace=True)
    
    # Remove any remaining columns that contain 'Unnamed'
    global_df = global_df.loc[:, ~global_df.columns.str.contains('Unnamed')]
    
    # Save the global dataframe to a CSV file for the current threshold
    global_df.to_csv(f'{path_results}displacement_risk_tc_global_admin0_{thresh}_check.csv', index=False)


In [13]:
for thresh in [0.3, 0.55, 0.7]:
    process_threshold(thresh, all_iso3, path_results)

### Rename admin1 to GADM classifications for better map plotting

In [8]:
import os
import re
import pandas as pd

gadm_attrs = pd.read_csv('/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk/data/shapefiles/attribute_table_gadm41.csv')
gadm_attrs_map = dict(zip(gadm_attrs.fid, gadm_attrs.GID_1))
gadm_attrs_map[-999] = 'admin0'
    
path_results = '/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk/data/results/risk_tc'

iso3_pattern = re.compile(r'^[A-Z]{3}$')
isos3 = os.listdir(path_results)

for iso3 in isos3:
    if not iso3_pattern.match(iso3):
        continue
    result_paths = os.listdir(path_results + '/' + iso3)
    
    for result_path in result_paths:
        try:
            # Load, modify, and save the CSV files
            df_results = pd.read_csv(path_results + '/' + iso3 + '/' + result_path)
            df_results.replace({'admin0': 'admin_-999'}, inplace=True)
            df_results['admin1'] = df_results.Exposure.apply(lambda x: x.split('_')[-1]).astype(int)
            df_results['GID_1'] = df_results.admin1.map(gadm_attrs_map)
            df_results.to_csv(path_results + '/' + iso3 + '/' + result_path)
        except Exception as e:
            print(f"Error processing {iso3}/{result_path}: {e}")

Error processing SVN/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing PCN/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing MNE/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing POL/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing MNP/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing BIH/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing GUM/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing FIN/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing SMR/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing UKR/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing ARG/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing ALB/note.txt: 'DataFrame' object has no attribute 'Exposure'
Error processing LVA/note.txt: 'DataFrame' object has no attribu

## Generate one single csv-File for admin-1

In [73]:
isos3 = os.listdir(path_results)

In [74]:
ssps = ['ssp245', 'ssp370','ssp585']

dict_scen = {'ssp245':'optimistic',
             'ssp370' : 'medium',
             'ssp585' : 'pessimistic',
             'ERA-5' : 'current'
            }

threshs = [0.3, 0.55, 0.7]
dict_thresh = {0.3 : 'high',
               0.55 : 'medium',
               0.7 : 'low'}


In [75]:
df_list = []
for iso3 in isos3:
        for ssp in ssps:
                try:
                    df03 = pd.read_csv(f'{path_results}/{iso3}/{iso3}_{ssp}_0.3_TC.csv')
                    df03 = df03.iloc[3:,:]
                    for col in df03.columns:
                        df03.rename({col:f'{col}_{dict_thresh[0.3]}'}, axis=1, inplace=True)
                    
                    df05 = pd.read_csv(f'{path_results}/{iso3}/{iso3}_{ssp}_0.55_TC.csv')
                    df05 = df05.iloc[3:,:]
                    for col in df05.columns:
                        df05.rename({col:f'{col}_{dict_thresh[0.55]}'}, axis=1, inplace=True)
                        
                    df07 = pd.read_csv(f'{path_results}/{iso3}/{iso3}_{ssp}_0.7_TC.csv')
                    df07 = df07.iloc[3:,:]
                    for col in df07.columns:
                        df07.rename({col:f'{col}_{dict_thresh[0.7]}'}, axis=1, inplace=True)
                except:
                    continue
                df_iso = pd.concat([df03[['Exposure_high','Period_high','Scenario_high','AAD_high','RP_10_high','RP_25_high','RP_50_high','RP_100_high','RP_250_high','admin1_high','GID_1_high']],
                                       df05[['Exposure_medium','Period_medium','Scenario_medium','AAD_medium','RP_10_medium','RP_25_medium','RP_50_medium','RP_100_medium','RP_250_medium']],
                                        df07[['Exposure_low','Period_low','Scenario_low','AAD_low','RP_10_low','RP_25_low','RP_50_low','RP_100_low','RP_250_low']]], 
                                                 axis=1)
                if 'GID_1' in df_iso.columns:
                    df_iso['GID_1'] = df_iso.GID_1.fillna(iso3)
                df_list.append(df_iso)


In [76]:
df_concat = pd.concat(df_list)

In [77]:
# delete unncecessary columns:
for col in df_concat:
    if 'Unnamed' in col:
        df_concat.pop(col)
for col in ['Exposure_high', 'Exposure_low','Exposure_medium', 'GID_1_medium', 'Period_high', 'Period_medium', 'Scenario_high', 'Scenario_medium']:
    if col in df_concat.columns:
        df_concat.pop(col)

In [78]:
df_concat

,AAD_high,RP_10_high,RP_25_high,RP_50_high,RP_100_high,RP_250_high,admin1_high,GID_1_high,AAD_medium,RP_10_medium,...,RP_100_medium,RP_250_medium,Period_low,Scenario_low,AAD_low,RP_10_low,RP_25_low,RP_50_low,RP_100_low,RP_250_low
3,0.0,0.0,0.0,0.0,0.0,0.0,435,CAN.2_1,0.0,0.0,...,0.0,0.0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,435,CAN.2_1,0.0,0.0,...,0.0,0.0,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,435,CAN.2_1,0.0,0.0,...,0.0,0.0,2100,ssp245,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,437,CAN.4_1,0.0,0.0,...,0.0,0.0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,437,CAN.4_1,0.0,0.0,...,0.0,0.0,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.0,0.0,0.0,0.0,0.0,0.0,3503,VEN.21_1,0.0,0.0,...,0.0,0.0,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,3503,VEN.21_1,0.0,0.0,...,0.0,0.0,2100,ssp585,0.0,0.0,0.0,0.0,0.0,0.0
126,0.0,0.0,0.0,0.0,0.0,0.0,3507,VEN.25_1,0.0,0.0,...,0.0,0.0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0
127,0.0,0.0,0.0,0.0,0.0,0.0,3507,VEN.25_1,0.0,0.0,...,0.0,0.0,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
df_concat.rename({'admin1_high':'fid'}, axis=1, inplace=True)
df_concat.rename({'GID_1_high':'GID_1'}, axis=1, inplace=True)
df_concat.rename({'Scenario_low':'scenario'}, axis=1, inplace=True)
df_concat.rename({'Period_low':'time_horzion'}, axis=1, inplace=True)

In [80]:
df_concat.replace(dict_scen, inplace=True)

In [81]:
for col in df_concat.columns:
    if 'RP' in col:
        rp, vuln = col.split('_')[-2:]
        df_concat.rename({col:f'PMD_{rp}_{vuln}'}, axis=1, inplace=True)
df_concat.columns

Index(['AAD_high', 'PMD_10_high', 'PMD_25_high', 'PMD_50_high', 'PMD_100_high',
       'PMD_250_high', 'fid', 'GID_1', 'AAD_medium', 'PMD_10_medium',
       'PMD_25_medium', 'PMD_50_medium', 'PMD_100_medium', 'PMD_250_medium',
       'time_horzion', 'scenario', 'AAD_low', 'PMD_10_low', 'PMD_25_low',
       'PMD_50_low', 'PMD_100_low', 'PMD_250_low'],
      dtype='object')

In [82]:
df_concat = df_concat[['fid', 'GID_1','time_horzion', 'scenario','AAD_high', 'PMD_10_high', 'PMD_25_high', 'PMD_50_high', 'PMD_100_high',
       'PMD_250_high', 'AAD_medium', 'PMD_10_medium',
       'PMD_25_medium', 'PMD_50_medium', 'PMD_100_medium', 'PMD_250_medium',
        'AAD_low', 'PMD_10_low', 'PMD_25_low',
       'PMD_50_low', 'PMD_100_low', 'PMD_250_low']]

In [83]:
df_concat

,fid,GID_1,time_horzion,scenario,AAD_high,PMD_10_high,PMD_25_high,PMD_50_high,PMD_100_high,PMD_250_high,...,PMD_25_medium,PMD_50_medium,PMD_100_medium,PMD_250_medium,AAD_low,PMD_10_low,PMD_25_low,PMD_50_low,PMD_100_low,PMD_250_low
3,435,CAN.2_1,hist,current,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,435,CAN.2_1,2050,optimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,435,CAN.2_1,2100,optimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,437,CAN.4_1,hist,current,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,437,CAN.4_1,2050,optimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,3503,VEN.21_1,2050,pessimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,3503,VEN.21_1,2100,pessimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,3507,VEN.25_1,hist,current,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,3507,VEN.25_1,2050,pessimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
# Some Admin1s are present several times due to pixel & border resolution --> sum them!

In [85]:
import copy as cp

#df_admin1_global = pd.read_csv('/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk/data/results/displacement_risk_tc_global_admin1_check.csv')
df_admin1_global = cp.deepcopy(df_concat)
#df_admin1_global.pop('Unnamed: 0')
#df_admin1_global

In [86]:
df_admin1_global.rename({'time_horzion':'time_horizon'}, axis=1, inplace=True)

In [87]:
df_admin1_global = df_admin1_global.groupby(['fid','GID_1','scenario','time_horizon']).sum()

In [88]:
df_admin1_global.reset_index()

,fid,GID_1,scenario,time_horizon,AAD_high,PMD_10_high,PMD_25_high,PMD_50_high,PMD_100_high,PMD_250_high,...,PMD_25_medium,PMD_50_medium,PMD_100_medium,PMD_250_medium,AAD_low,PMD_10_low,PMD_25_low,PMD_50_low,PMD_100_low,PMD_250_low
0,1,AFG.1_1,current,hist,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,AFG.1_1,medium,2050,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,AFG.1_1,medium,2100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,AFG.1_1,optimistic,2050,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,AFG.1_1,optimistic,2100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17843,3662,ZWE.10_1,medium,2100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17844,3662,ZWE.10_1,optimistic,2050,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17845,3662,ZWE.10_1,optimistic,2100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17846,3662,ZWE.10_1,pessimistic,2050,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
#df_admin1_global.to_csv('/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk/data/results/displacement_risk_tc_global_admin1_check.csv')